This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [40]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Mount Google Drive (Because Colab is working from the cloud)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting Directories


In [41]:
TRAIN_directory = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_directory = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'

## Cleaning & Preparing the Data

Finding 3000 most common words
 



---



This function builds a Dictionary of most common 3000 words from all the 
email content. It eliminates "stop words" such as is, this, are, that doen contribute much to the anlaysis.

First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [42]:
def my_dictionary(main_directory):
    # Create dictionary called make_Dictionary that looks for 'root_dir'
  word_list = []
    # Makes a blank list to store the words in later
  email_list = [os.path.join(main_directory,f) for f in os.listdir(main_directory)]
    # Joins path components. Concatenates various path compnents with one directory separator
    # os.path.join(path, *paths) ; joins one or more path components/directories 
    # f = number of files
    # emails is a list of all of the files
  for message in email_list:
    # Iterates through every element in emails
    with open(message) as m:
      # Opens each email, defined as 'm'
      # Looks through each email independently and closes it after
      for each_line in m:
      # looks through each line in each email from "emails"
        email_text = each_line.split()
      # splits the email at the spaces to extract each word
        word_list += email_text
      # Adds a value and a variable and assigns the results to that variable
  dictionary = Counter(word_list)
      # is a container that keeps track of how many times equivalent values are added. (Similar to value_counts or unique function)
  list_to_remove = list(dictionary)
      # Transforms the dictionary to a list 

  for wrd in list_to_remove:
      # looks through each word in the list ' list_to_remove'
    if wrd.isalpha() == False:
      # Checks for whether element is alphabetical or numerical
      del dictionary[wrd]
      # if item is numeric, it will drop it from thex list
    elif len(wrd) == 1:
      del dictionary[wrd]
      # removes single character 'words'
  dictionary = dictionary.most_common(3000)
      # Extracts 3000 most common words
  return dictionary



In [ ]:
# Checking what emails list produces (Not required)
onedir = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
emails = [os.path.join(onedir,f) for f in os.listdir(onedir)]
emails

## Extracting Features and Label Matrix
Generates a label and word frequency matrix

---



This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files).

The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. 

This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [51]:
def extract_features(mail_directory):
  email_files = [os.path.join(mail_directory,fi) for fi in os.listdir(mail_directory)]
  # Joins path components. Concatenates various path compnents with one directory separator
  # os.path.join(path, *paths) ; joins one or more path components/directories
  # Joins TEST_directory (mail_directory) with fi (A list of the files inside TEST_directory)
  features_matrix = np.zeros((len(email_files),3000))
    # Returns a new  2D array with shape of 3000 columns, rows = to the length of the number of emails titled 'features_matrix'
  train_labels = np.zeros(len(email_files))    
    # 1D array with # of column as the length of files
  spam_numb = 0;  
  docID = 0;
  for one_file in email_files:
    with open(one_file) as fi:
      # opens each file independently; returns it as a file object
      for i, line in enumerate(fi):
        # Enumerate returns an index number and the value
        if i ==2:
          # Looks to the 3rd line of the file because first two lines are irrelevant (Subject and blank).
          email_text = line.split()
          # Splits each value/word at the spaces to separate each word
          for word in email_text:
            wordID = 0
            # Setting a base word ID to 0 
            for i, d in enumerate(dictionary):
              if d[0] == word:
                # Confirming if the word in the loop is in the dictionary
                # If the word is in the dictionary (previously created in the fucntion above), it will be added to the matrix
                wordID = i
                # 0 is no spam, # 1 is spam, evaluated on naming convention

                features_matrix[docID,wordID] = email_text.count(word)
      train_labels[docID] = 0;
          # Setting a base docID if emial is not spam
      filepathTokens = one_file.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
          # If the last work has "spmsg", it will be identified as spam and assigned 1
        train_labels[docID] = 1;
          # if the email is identified as spam, it is reassigned to 1
        spam_numb = spam_numb + 1
          # spam_numb will count the number of spam messages by adding 1 every time it finds a spam message
      docID = docID + 1
          # Creating a new docID for non-spam emails
  return features_matrix, train_labels



## Running the previous functions & printing accuracy scores

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [53]:
dictionary = my_dictionary(TRAIN_directory)
  # Creating 3000 most common words dictionary from the training data

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_directory)
test_features_matrix, test_labels = extract_features(TEST_directory)

model = GaussianNB()
  # Setting model to run the Gaussian Naive Bayes model. This accoutns for normal distribution and supports contonuous data

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels)  
  # Essentially trains the model. Fits it with a features column and labels column
print ("Training completed")
  # Prints Training Completed
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))
  # Runs accuracy score based on predicted labels from the model and actual labels

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154


======================= END OF PROGRAM =========================